# Installation

In [1]:
!pip install scikit-learn
!pip install pythainlp
!pip install emoji
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.2 MB/s eta 0:00:00


In [2]:
import os

# Create a directory for the downloaded dataset.
dataset_name = "wongnai-dataset"
os.makedirs(dataset_name, exist_ok=True)

# Download the dataset from google drive.
!wget https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip

# Unzip the dataset.
!unzip review_dataset.zip -d wongnai-dataset # for linux
# !tar -xzvf review_dataset.zip -C wongnai-dataset # for windows

# Remove the zip file.
!rm review_dataset.zip
# Remove the unrelated __MACOSX folder.
!rm -r wongnai-dataset/__MACOSX

--2023-08-26 15:44:20--  https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip [following]
--2023-08-26 15:44:20--  https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14338706 (14M) [application/zip]
Saving to: ‘review_dataset.zip’

review_dataset.zip  100%[===================>]  13.67M  --.-KB/s    in 0.08s   

2023-08-26 15:44:21 (167 MB/s) - ‘review_dataset.zip’ saved [1433870

# Data Preparation

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the train dataset.
train_df = pd.read_csv("wongnai-dataset/w_review_train.csv",
    sep=";",
    names=["review", "rating"],
    header=None
)
# Remove duplicate rows from training dataset.
train_df.drop_duplicates(inplace=True)

# Read the test dataset.
test_df = pd.read_csv("wongnai-dataset/test_file.csv", sep=";")
test_df["rating"] = 0

In [5]:
# Preview the percentage of each rating in training dataset.
total_rating_samples = train_df.shape[0]
train_df.rating.value_counts() / total_rating_samples

4    0.469282
3    0.304328
5    0.169880
2    0.046133
1    0.010377
Name: rating, dtype: float64

In [6]:
# Let's duplicate data on "1" and "2" rating, because it has small samples.
two_df = pd.concat([train_df[train_df.rating==2].copy() for i in range(2)]).reset_index(drop=True)
one_df = pd.concat([train_df[train_df.rating==1].copy() for i in range(10)]).reset_index(drop=True)

# We create a train balance dataset by adding a duplicate "1" and "2" dataset.
train_bal = pd.concat([train_df, one_df, two_df]).reset_index(drop=True)

# Preview the percentage of each rating on train balance dataset.
total_rating_samples = train_bal.shape[0]
train_bal.rating.value_counts() / total_rating_samples

4    0.392365
3    0.254448
5    0.142036
2    0.115715
1    0.095436
Name: rating, dtype: float64

In [7]:
# Get each dataset components.
X_train, y_train = train_bal["review"], train_bal["rating"]
X_test = test_df["review"]

# Training Model

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from pythainlp.ulmfit import process_thai

# Create a pipeline for text classification.
rating_classifier = Pipeline([
    ("vect", CountVectorizer(tokenizer=process_thai, ngram_range=(1, 2))),
    ("tfidf", TfidfTransformer()),
    ("clf", LinearSVC()),
])

In [13]:
# Train a classifier.
rating_classifier.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function process_thai at 0x7f848209be20>)),
                ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])

# Predict

In [14]:
# Predict test dataset to get rating predictions
rating_predictions = rating_classifier.predict(X_test)

In [50]:
# Create a prediction result dataframe.
submit_df = pd.DataFrame({
    "review": test_df.review,
    "rating": rating_predictions
})
# See the first 10 results
pd.set_option('display.max_colwidth', 150) # Make long text easier to read
submit_df.head()

,review,rating
0,"ร้านนี้จะอยู่เส้นสันกำแพง-แม่ออน เลยแยกบ่อสร้างร้านจะอยู่ด้านซ้ายติดริมถนน มีป้ายติดไว้เห็นชัดเจน..""ปูทอง ข้าวแกงรสเด็ด""\n ตามหาข้าวแกงรส...",4
1,สั่งไป2 เมนู คือมัชฉะลาเต้ร้อน กับ ไอศครีมชาเขียว มัชฉะลาเต้ร้อน รสชาเขียวเข้มข้น หอม มัน แต่ไม่กลมกล่อม มันจืดแบบจืดสนิท ส่วนไอศครีมชาเขียว ทานแล...,3
2,ครัววงเดือน \n\nหิวดึกๆ ตระเวนหาร้านทาน มาเจอร้านริมถนนพุทธมณฑลสาย 1 หน้าปากซอยพุทธมณฑลสาย 1 ซอย 10ครับ ร้านวงเดือน ขายทั้งอาหารไทยทั่วไป ทะเลเผา...,3
3,จะว่าเป็นเจ้าประจำก็คงไม่ผิด แต่ก็ไม่กล้าการันตีหรอกว่าร้านนี้อร่อยที่สุดของหล่มเก่า รู้แต่เพียงว่าเป็นเจ้าแรก ๆ และขายมานาน ผ่านการแนะนำของค...,4
4,ถ้าคิดถึงสลัดผมคิดถึงร้านนี้เป็นร้านแรกๆเลยครับ\nเพราะมีให้เลือกหลากหลายแนว\nอยากกินสลัดแนวไหนนี่มีครบ\nไม่ว่าจะ healthy หรือว่าแบบครีมๆจัดเต็มก็ม...,3


In [40]:
# Define function for later predict.
def predict_rating(review: str) -> str:
  """Predict a rating of a given text review."""
  predictions = rating_classifier.predict([review])
  prediction = predictions[0]
  return str(prediction)

# Try bad rating prediction
bad_review_text = "อาหารแย่มากๆ ไม่อร่อยเลย บรรยากาศร้านสกปรกและไม่ดูเอาจริงในราคาที่แพงมาก พนักงานบริการก็ไม่ใส่ใจเลย ไม่แนะนำเลยค่ะ อย่าไปเสียเวลาและเงินกับร้านนี้"
predicted_rating = predict_rating(bad_review_text)

print(f"Review: {bad_review_text}")
print(f"Predicted rating: {predicted_rating}")

# Try good rating prediction
good_review_text = "อาหารอร่อยมากค่ะ! บรรยากาศร้านสวยงามและเป็นกันเอง พนักงานบริการดีมาก ไม่เคยผิดหวังเลย ขอแนะนำเมนูทานเล่นและสเต็กที่นี่นะคะ สั่งมาทานหลายครั้งแล้วครับ ถ้ามีโอกาสจะกลับมาใหม่แน่นอน!"
predicted_rating = predict_rating(good_review_text)

print(f"Review: {good_review_text}")
print(f"Predicted rating: {predicted_rating}")

Review: อาหารแย่มากๆ ไม่อร่อยเลย บรรยากาศร้านสกปรกและไม่ดูเอาจริงในราคาที่แพงมาก พนักงานบริการก็ไม่ใส่ใจเลย ไม่แนะนำเลยค่ะ อย่าไปเสียเวลาและเงินกับร้านนี้
Predicted rating: 1
Review: อาหารอร่อยมากค่ะ! บรรยากาศร้านสวยงามและเป็นกันเอง พนักงานบริการดีมาก ไม่เคยผิดหวังเลย ขอแนะนำเมนูทานเล่นและสเต็กที่นี่นะคะ สั่งมาทานหลายครั้งแล้วครับ ถ้ามีโอกาสจะกลับมาใหม่แน่นอน!
Predicted rating: 5


# Gradio Interface

In [41]:
from gradio.components import Textbox, Label
from gradio import Interface

# Create a gradio interface
rating_interface = Interface(
    fn=predict_rating,
    inputs=Textbox(label="Review"),
    outputs=Label(label="Predicted Rating")
)
# Launch the webapp
rating_interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>